# BCI Training & Fine-Tuning Pipeline Test - Google Colab Version

Este notebook testa o pipeline completo no Google Colab:
1. **Setup**: Instalar dependências e clonar repositório
2. **Treinamento Principal**: Subjects 1-79 do dataset PhysioNet
3. **Fine-Tuning**: Dados específicos da pasta Davi
4. **Validação**: Métricas e comparação de performance

## ⚠️ IMPORTANTE: Este notebook foi adaptado para Google Colab

## 1. Setup Inicial no Google Colab

Vamos instalar as dependências e configurar o ambiente adequadamente.

In [ ]:
# === INSTALAÇÃO DE DEPENDÊNCIAS ===
print("🔧 Instalando dependências necessárias...")

# Instalar braindecode e dependências científicas
!pip install braindecode mne torch torchvision torchaudio
!pip install scikit-learn pandas numpy matplotlib seaborn
!pip install scipy jupyter

print("✅ Dependências instaladas!")

# Verificar se está no Colab
try:
    import google.colab
    IN_COLAB = True
    print("📱 Executando no Google Colab")
except ImportError:
    IN_COLAB = False
    print("💻 Executando localmente")

In [ ]:
# === CLONE DO REPOSITÓRIO (OPCIONAL) ===
import os
from pathlib import Path

if IN_COLAB:
    print("🔄 Configurando para Google Colab...")
    
    # Se você quiser clonar seu repositório:
    # !git clone https://github.com/seu-usuario/projetoBCI.git
    # os.chdir('/content/projetoBCI')
    
    # Por enquanto, vamos trabalhar no diretório padrão do Colab
    project_root = Path('/content')
    
    # Criar estrutura de diretórios
    (Path("/content/drive/MyDrive/Colab Notebooks/eeg_data")).mkdir(exist_ok=True)
    (project_root / "models").mkdir(exist_ok=True)
    (project_root / "results").mkdir(exist_ok=True)
    
else:
    # Executando localmente
    project_root = Path(os.getcwd()).parent

print(f"📂 Diretório de trabalho: {project_root}")
print(f"📁 Estrutura criada!")

## 2. Implementação das Classes Base (Inline)

Já que não temos o repositório clonado, vamos implementar as classes principais inline.

In [ ]:
# === IMPLEMENTAÇÃO INLINE DAS CLASSES BASE ===
import torch
import torch.nn as nn
import numpy as np
from scipy import signal
from typing import Optional, List, Tuple, Dict, Union
import pandas as pd
from datetime import datetime
import json
import logging
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Imports básicos carregados!")

In [ ]:
# === UNIVERSAL EEG NORMALIZER ===
class UniversalEEGNormalizer:
    """Normalizador universal para dados EEG"""
    
    def __init__(self, method: str = 'zscore', mode: str = 'training'):
        self.method = method
        self.mode = mode
        self.global_stats = {}
        self.is_fitted = False
    
    def _ensure_3d(self, data: np.ndarray) -> np.ndarray:
        """Garantir que dados estejam em 3D (n_samples, n_channels, n_timepoints)"""
        if len(data.shape) == 2:
            n_samples, n_features = data.shape
            if n_features % 16 == 0:  # Assumir 16 canais
                n_channels = 16
                n_timepoints = n_features // n_channels
                data = data.reshape(n_samples, n_channels, n_timepoints)
            else:
                data = data[:, np.newaxis, :]
        elif len(data.shape) == 1:
            data = data[np.newaxis, np.newaxis, :]
        return data
    
    def fit(self, data: np.ndarray):
        """Ajustar normalizador aos dados"""
        data_3d = self._ensure_3d(data)
        
        if self.method == 'zscore':
            self.global_stats['mean'] = np.mean(data_3d, axis=(0, 2), keepdims=True)
            self.global_stats['std'] = np.std(data_3d, axis=(0, 2), keepdims=True)
            self.global_stats['std'] = np.where(self.global_stats['std'] == 0, 1.0, self.global_stats['std'])
        
        self.is_fitted = True
        return self
    
    def transform(self, data: np.ndarray) -> np.ndarray:
        """Transformar dados"""
        if not self.is_fitted:
            raise ValueError("Normalizador deve ser ajustado antes da transformação")
        
        original_shape = data.shape
        data_3d = self._ensure_3d(data)
        
        if self.method == 'zscore':
            normalized = (data_3d - self.global_stats['mean']) / self.global_stats['std']
        
        # Restaurar forma original
        if len(original_shape) == 2:
            return normalized.reshape(original_shape[0], -1)
        elif len(original_shape) == 1:
            return normalized.flatten()
        return normalized
    
    def fit_transform(self, data: np.ndarray) -> np.ndarray:
        """Ajustar e transformar em um passo"""
        return self.fit(data).transform(data)

print("✅ UniversalEEGNormalizer implementado!")

In [ ]:
# === BCI DATASET CLASS ===
class BCIDataset(Dataset):
    """Dataset PyTorch para dados EEG"""
    
    def __init__(self, windows: np.ndarray, labels: np.ndarray, transform=None, augment: bool = False):
        self.windows = torch.from_numpy(windows).float()
        self.labels = torch.from_numpy(labels).long()
        self.transform = transform
        self.augment = augment
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        window = self.windows[idx]
        label = self.labels[idx]
        
        if self.augment:
            # Adicionar ruído leve
            if torch.rand(1) < 0.3:
                noise = torch.randn_like(window) * 0.01
                window = window + noise
        
        if self.transform:
            window = self.transform(window)
        
        return window, label

print("✅ BCIDataset implementado!")

In [ ]:
# === EEG INCEPTION ERP MODEL ===
try:
    from braindecode.models import EEGInceptionERP
    print("✅ Braindecode EEGInceptionERP importado com sucesso!")
    BRAINDECODE_AVAILABLE = True
except ImportError as e:
    print(f"❌ Erro ao importar braindecode: {e}")
    print("🔧 Vamos implementar um modelo CNN simples como fallback")
    BRAINDECODE_AVAILABLE = False

class EEGInceptionERPModel(nn.Module):
    """Wrapper para EEGInceptionERP com fallback para CNN simples"""
    
    def __init__(self, n_chans: int, n_outputs: int, n_times: int, sfreq: float = 125.0, **kwargs):
        super().__init__()
        self.n_chans = n_chans
        self.n_outputs = n_outputs
        self.n_times = n_times
        self.sfreq = sfreq
        self.is_trained = False
        
        if BRAINDECODE_AVAILABLE:
            try:
                self.model = EEGInceptionERP(
                    n_chans=n_chans,
                    n_outputs=n_outputs,
                    n_times=n_times,
                    sfreq=sfreq
                )
                self.model_type = "EEGInceptionERP"
                print(f"✅ Usando EEGInceptionERP: {n_chans} canais, {n_times} pontos temporais")
            except Exception as e:
                print(f"⚠️ Erro ao criar EEGInceptionERP: {e}")
                self.model = self._create_fallback_cnn(n_chans, n_outputs, n_times)
                self.model_type = "FallbackCNN"
        else:
            self.model = self._create_fallback_cnn(n_chans, n_outputs, n_times)
            self.model_type = "FallbackCNN"
    
    def _create_fallback_cnn(self, n_chans: int, n_outputs: int, n_times: int):
        """Criar CNN simples como fallback"""
        print(f"🔧 Criando CNN fallback: {n_chans} canais, {n_times} pontos temporais")
        return nn.Sequential(
            # Convolução temporal
            nn.Conv1d(n_chans, 32, kernel_size=25, padding=12),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            # Redução de dimensionalidade
            nn.Conv1d(32, 64, kernel_size=15, padding=7),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(4),
            nn.Dropout(0.25),
            
            # Convolução final
            nn.Conv1d(64, 128, kernel_size=7, padding=3),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            
            # Classificador
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, n_outputs)
        )
    
    def forward(self, x):
        if self.model_type == "EEGInceptionERP":
            return self.model(x)
        else:
            # Para o CNN fallback, precisamos transpor de (batch, channels, time) para (batch, channels, time)
            return self.model(x)
    
    def save(self, filepath: str):
        """Salvar modelo"""
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        save_dict = {
            'model_state_dict': self.model.state_dict(),
            'model_type': self.model_type,
            'n_chans': self.n_chans,
            'n_outputs': self.n_outputs,
            'n_times': self.n_times,
            'sfreq': self.sfreq,
            'is_trained': self.is_trained
        }
        torch.save(save_dict, filepath)
        print(f"✅ Modelo salvo: {filepath}")
    
    def load(self, filepath: str):
        """Carregar modelo"""
        checkpoint = torch.load(filepath, map_location='cpu')
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.is_trained = checkpoint.get('is_trained', True)
        print(f"✅ Modelo carregado: {filepath}")

print(f"✅ EEGInceptionERPModel implementado! Tipo: {'Braindecode' if BRAINDECODE_AVAILABLE else 'Fallback CNN'}")

## 3. Configuração dos Caminhos e Parâmetros

In [ ]:
# Configuração dos caminhos para Colab
EEG_DATA_PATH = Path("/content/drive/MyDrive/Colab Notebooks/eeg_data")
DAVI_DATA_PATH = EEG_DATA_PATH / "Davi"
MODELS_PATH = project_root / "models"
RESULTS_PATH = project_root / "results"

# Parâmetros de treinamento (ajustados para teste rápido no Colab)
MAIN_TRAINING_PARAMS = {
    "subjects_to_use": [1, 2, 3],  # TESTE: apenas 3 subjects
    "num_epochs_per_fold": 5,  # TESTE: apenas 5 epochs
    "num_k_folds": 3,  # TESTE: apenas 3 folds
    "learning_rate": 0.001,
    "early_stopping_patience": 3,
    "batch_size": 8,  # TESTE: batch menor
    "test_split_ratio": 0.2,
    "data_base_path": str(EEG_DATA_PATH),
    "model_name": "colab_test_model"
}

# Parâmetros de fine-tuning
FINE_TUNING_PARAMS = {
    "freeze_strategy": "early",
    "learning_rate_ratio": 0.1,
    "epochs": 10,  # TESTE: poucos epochs
    "batch_size": 8,
    "validation_split": 0.2
}

print("✅ Configurações definidas para Google Colab!")
print(f"📂 Diretório de dados: {EEG_DATA_PATH}")
print(f"🏗️ Modelos: {MODELS_PATH}")
print(f"📊 Resultados: {RESULTS_PATH}")
print("\n⚠️ NOTA: Parâmetros reduzidos para teste rápido no Colab")

## 4. Geração de Dados Sintéticos para Teste

Já que provavelmente você não tem os dados reais no Colab, vamos gerar dados sintéticos para testar o pipeline.

In [ ]:
# === GERAÇÃO DE DADOS SINTÉTICOS ===
def generate_synthetic_/content/drive/MyDrive/Colab Notebooks/eeg_data(n_subjects: int = 3, n_sessions_per_subject: int = 3):
    """Gerar dados EEG sintéticos para teste"""
    print(f"🎲 Gerando dados sintéticos: {n_subjects} subjects, {n_sessions_per_subject} sessões cada")
    
    # Parâmetros dos dados
    n_channels = 16
    n_timepoints = 400  # 3.2s a 125Hz
    n_trials_per_session = 50
    
    all_windows = []
    all_labels = []
    all_subject_ids = []
    
    for subject_id in range(1, n_subjects + 1):
        for session in range(n_sessions_per_subject):
            # Gerar dados com padrões diferentes por classe
            for trial in range(n_trials_per_session):
                # Classe aleatória (0: mão esquerda, 1: mão direita)
                label = np.random.randint(0, 2)
                
                # Gerar sinal base
                window = np.random.randn(n_channels, n_timepoints) * 0.1
                
                # Adicionar padrão específico da classe
                if label == 0:  # Mão esquerda - mais atividade em C3 (canal 7)
                    window[7] += np.sin(np.linspace(0, 4*np.pi, n_timepoints)) * 0.3
                    window[8] += np.cos(np.linspace(0, 3*np.pi, n_timepoints)) * 0.2
                else:  # Mão direita - mais atividade em C4 (canal 9)
                    window[9] += np.sin(np.linspace(0, 4*np.pi, n_timepoints)) * 0.3
                    window[10] += np.cos(np.linspace(0, 3*np.pi, n_timepoints)) * 0.2
                
                # Adicionar ruído específico do subject
                subject_noise = np.random.randn(n_channels, n_timepoints) * (0.05 + subject_id * 0.01)
                window += subject_noise
                
                all_windows.append(window)
                all_labels.append(label)
                all_subject_ids.append(subject_id)
    
    return np.array(all_windows), np.array(all_labels), np.array(all_subject_ids)

# Gerar dados sintéticos
print("🎲 Gerando dataset sintético...")
synthetic_windows, synthetic_labels, synthetic_subject_ids = generate_synthetic_/content/drive/MyDrive/Colab Notebooks/eeg_data()

print(f"✅ Dataset sintético criado:")
print(f"   - Formato: {synthetic_windows.shape}")
print(f"   - Labels: {np.bincount(synthetic_labels)}")
print(f"   - Subjects: {np.unique(synthetic_subject_ids)}")

# Salvar dados sintéticos
synthetic_data_path = EEG_DATA_PATH / "synthetic_data.npz"
EEG_DATA_PATH.mkdir(exist_ok=True)
np.savez(synthetic_data_path, 
         windows=synthetic_windows, 
         labels=synthetic_labels, 
         subject_ids=synthetic_subject_ids)
print(f"💾 Dados sintéticos salvos em: {synthetic_data_path}")

## 5. Implementação do Pipeline de Treinamento Simplificado

In [ ]:
# === PIPELINE DE TREINAMENTO SIMPLIFICADO ===
def train_simplified_model(windows, labels, subject_ids, params):
    """Pipeline de treinamento simplificado para Colab"""
    
    print("🚀 Iniciando treinamento simplificado...")
    
    # Filtrar subjects
    if isinstance(params["subjects_to_use"], list):
        mask = np.isin(subject_ids, params["subjects_to_use"])
        windows = windows[mask]
        labels = labels[mask]
        subject_ids = subject_ids[mask]
    
    print(f"📊 Dados filtrados: {windows.shape[0]} amostras")
    
    # Normalizar dados
    normalizer = UniversalEEGNormalizer(method='zscore')
    windows_norm = normalizer.fit_transform(windows)
    
    # Split test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        windows_norm, labels, 
        test_size=params["test_split_ratio"], 
        random_state=42, 
        stratify=labels
    )
    
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🖥️ Usando device: {device}")
    
    # K-fold CV
    kfold = KFold(n_splits=params["num_k_folds"], shuffle=True, random_state=42)
    fold_results = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_val), 1):
        print(f"\n📁 Treinando fold {fold}/{params['num_k_folds']}...")
        
        # Split fold
        X_train_fold = X_train_val[train_idx]
        X_val_fold = X_train_val[val_idx]
        y_train_fold = y_train_val[train_idx]
        y_val_fold = y_train_val[val_idx]
        
        # Datasets
        train_dataset = BCIDataset(X_train_fold, y_train_fold, augment=True)
        val_dataset = BCIDataset(X_val_fold, y_val_fold, augment=False)
        
        train_loader = DataLoader(train_dataset, batch_size=params["batch_size"], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=params["batch_size"], shuffle=False)
        
        # Modelo
        model = EEGInceptionERPModel(
            n_chans=windows.shape[1],
            n_outputs=len(np.unique(labels)),
            n_times=windows.shape[2]
        ).to(device)
        
        # Treinamento
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=params["learning_rate"])
        
        best_val_acc = 0.0
        patience_counter = 0
        
        for epoch in range(params["num_epochs_per_fold"]):
            # Training
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0
            
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                train_total += batch_y.size(0)
                train_correct += (predicted == batch_y).sum().item()
            
            # Validation
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            
            with torch.no_grad():
                for batch_x, batch_y in val_loader:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                    outputs = model(batch_x)
                    loss = criterion(outputs, batch_y)
                    
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += batch_y.size(0)
                    val_correct += (predicted == batch_y).sum().item()
            
            train_acc = train_correct / train_total
            val_acc = val_correct / val_total
            
            print(f"  Epoch {epoch+1}: Train Acc: {train_acc:.3f}, Val Acc: {val_acc:.3f}")
            
            # Early stopping
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience_counter = 0
                # Salvar melhor modelo do fold
                model_path = MODELS_PATH / params["model_name"] / f"fold_{fold}.pth"
                model_path.parent.mkdir(parents=True, exist_ok=True)
                model.save(str(model_path))
            else:
                patience_counter += 1
                if patience_counter >= params["early_stopping_patience"]:
                    print(f"  Early stopping em epoch {epoch+1}")
                    break
        
        fold_results.append(best_val_acc)
        print(f"✅ Fold {fold} concluído - Melhor Val Acc: {best_val_acc:.4f}")
    
    # Treinamento final
    print(f"\n🎯 Treinamento final em todos os dados...")
    final_dataset = BCIDataset(X_train_val, y_train_val, augment=True)
    test_dataset = BCIDataset(X_test, y_test, augment=False)
    
    final_train_loader = DataLoader(final_dataset, batch_size=params["batch_size"], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=params["batch_size"], shuffle=False)
    
    final_model = EEGInceptionERPModel(
        n_chans=windows.shape[1],
        n_outputs=len(np.unique(labels)),
        n_times=windows.shape[2]
    ).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(final_model.parameters(), lr=params["learning_rate"])
    
    # Treinar modelo final
    for epoch in range(params["num_epochs_per_fold"]):
        final_model.train()
        for batch_x, batch_y in final_train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = final_model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    # Testar modelo final
    final_model.eval()
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = final_model(batch_x)
            _, predicted = torch.max(outputs.data, 1)
            test_total += batch_y.size(0)
            test_correct += (predicted == batch_y).sum().item()
    
    final_test_acc = test_correct / test_total
    
    # Salvar modelo final
    final_model_path = MODELS_PATH / params["model_name"] / "final_model.pth"
    final_model.save(str(final_model_path))
    final_model.is_trained = True
    
    # Resultados
    cv_mean = np.mean(fold_results)
    cv_std = np.std(fold_results)
    
    results = {
        'cv_mean_accuracy': cv_mean,
        'cv_std_accuracy': cv_std,
        'final_test_accuracy': final_test_acc,
        'model_name': params["model_name"],
        'fold_accuracies': fold_results
    }
    
    print(f"\n🎉 TREINAMENTO CONCLUÍDO!")
    print(f"   CV Mean: {cv_mean:.4f} ± {cv_std:.4f}")
    print(f"   Test Acc: {final_test_acc:.4f}")
    
    return results

print("✅ Pipeline de treinamento simplificado implementado!")

## 6. Execução do Treinamento Principal

In [ ]:
# === EXECUÇÃO DO TREINAMENTO ===
print("🚀 INICIANDO TREINAMENTO PRINCIPAL...")

try:
    # Executar treinamento com dados sintéticos
    main_results = train_simplified_model(
        synthetic_windows, 
        synthetic_labels, 
        synthetic_subject_ids, 
        MAIN_TRAINING_PARAMS
    )
    
    print("\n📊 RESULTADOS DO TREINAMENTO:")
    print(f"  - CV Mean: {main_results['cv_mean_accuracy']:.4f} ± {main_results['cv_std_accuracy']:.4f}")
    print(f"  - Test Acc: {main_results['final_test_accuracy']:.4f}")
    print(f"  - Modelo: {main_results['model_name']}")
    
    # Salvar resultados
    results_path = RESULTS_PATH / "main_training_results.json"
    RESULTS_PATH.mkdir(exist_ok=True)
    with open(results_path, 'w') as f:
        json.dump(main_results, f, indent=2, default=str)
    
    print(f"💾 Resultados salvos em: {results_path}")
    
except Exception as e:
    print(f"❌ Erro durante treinamento: {e}")
    import traceback
    traceback.print_exc()
    main_results = None

## 7. Visualização dos Resultados

In [ ]:
# === VISUALIZAÇÃO DOS RESULTADOS ===
if main_results:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('Resultados do Treinamento BCI - Google Colab', fontsize=16, fontweight='bold')
    
    # 1. Acurácias por fold
    ax1 = axes[0, 0]
    fold_accs = main_results['fold_accuracies']
    ax1.bar(range(1, len(fold_accs) + 1), fold_accs, alpha=0.7, color='skyblue')
    ax1.axhline(y=main_results['cv_mean_accuracy'], color='red', linestyle='--', 
                label=f"Mean: {main_results['cv_mean_accuracy']:.3f}")
    ax1.set_xlabel('Fold')
    ax1.set_ylabel('Acurácia')
    ax1.set_title('Acurácia por Fold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Comparação CV vs Test
    ax2 = axes[0, 1]
    models = ['CV Mean', 'Test Final']
    accuracies = [main_results['cv_mean_accuracy'], main_results['final_test_accuracy']]
    bars = ax2.bar(models, accuracies, color=['lightcoral', 'lightgreen'], alpha=0.8)
    ax2.set_ylabel('Acurácia')
    ax2.set_title('CV vs Test Performance')
    ax2.set_ylim(0, 1)
    
    for bar, acc in zip(bars, accuracies):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 3. Distribuição das classes sintéticas
    ax3 = axes[1, 0]
    unique, counts = np.unique(synthetic_labels, return_counts=True)
    ax3.pie(counts, labels=[f'Classe {u}' for u in unique], autopct='%1.1f%%')
    ax3.set_title('Distribuição de Classes')
    
    # 4. Resumo textual
    ax4 = axes[1, 1]
    ax4.axis('off')
    summary_text = f"""
    RESUMO DO TREINAMENTO
    
    Dataset: Sintético
    Subjects: {len(np.unique(synthetic_subject_ids))}
    Amostras: {len(synthetic_labels)}
    
    Modelo: {main_results['model_name']}
    Tipo: {'EEGInceptionERP' if BRAINDECODE_AVAILABLE else 'CNN Fallback'}
    
    Performance:
    • CV Mean: {main_results['cv_mean_accuracy']:.4f}
    • CV Std: {main_results['cv_std_accuracy']:.4f}
    • Test Acc: {main_results['final_test_accuracy']:.4f}
    
    Status: ✅ Sucesso
    """
    ax4.text(0.1, 0.9, summary_text, transform=ax4.transAxes, 
             fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.savefig(RESULTS_PATH / 'colab_training_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Visualização criada e salva!")
else:
    print("❌ Não há resultados para visualizar")

## 8. Relatório Final

Resumo da execução no Google Colab com implementação inline.

In [ ]:
print("=" * 60)
print("            RELATÓRIO FINAL - GOOGLE COLAB")
print("=" * 60)

print(f"\n🖥️ AMBIENTE:")
print(f"  - Plataforma: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"  - Braindecode: {'✅ Disponível' if BRAINDECODE_AVAILABLE else '❌ Fallback CNN'}")
print(f"  - Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

print(f"\n📊 DADOS:")
print(f"  - Tipo: Sintético")
print(f"  - Amostras: {len(synthetic_labels)}")
print(f"  - Subjects: {len(np.unique(synthetic_subject_ids))}")
print(f"  - Classes: {len(np.unique(synthetic_labels))}")

if main_results:
    print(f"\n🎯 RESULTADOS:")
    print(f"  - Modelo: {main_results['model_name']}")
    print(f"  - CV Accuracy: {main_results['cv_mean_accuracy']:.4f} ± {main_results['cv_std_accuracy']:.4f}")
    print(f"  - Test Accuracy: {main_results['final_test_accuracy']:.4f}")
    print(f"  - Status: ✅ SUCESSO")
else:
    print(f"\n❌ RESULTADOS: FALHOU")

print(f"\n📁 ARQUIVOS CRIADOS:")
for path in [MODELS_PATH, RESULTS_PATH]:
    if path.exists():
        files = list(path.rglob("*"))
        print(f"  - {path.name}/: {len(files)} arquivo(s)")

print(f"\n🎉 CONCLUSÃO:")
print(f"  Este notebook demonstrou como adaptar o pipeline BCI")
print(f"  para execução no Google Colab, incluindo:")
print(f"  • Implementação inline das classes principais")
print(f"  • Geração de dados sintéticos para teste")
print(f"  • Pipeline de treinamento simplificado")
print(f"  • Fallback para quando braindecode não está disponível")

print(f"\n✨ Execução concluída em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)